# Proyecto - Deteccion y mitigacion de ataques DDoS en redes IoT/Cloud usando ML/DL

## Paso 1. Preprocesamiento de datos

El preprocesamiento de datos es una etapa **esencial** en la construcción de
modelos de ML/DL. En esta etapa, se realizan tareas como la limpieza de
datos, la transformación de variables, la selección de características, con
el fin de preparar los datos para el entrenamiento de los modelos propuestos.

---

### Importar librerias necesarias

In [ ]:
# Importamos librerias necesarias
import pandas as pd
from dotenv import load_dotenv
from os.path import join as pjoin
from os import listdir
from os import getenv

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

# Cargamos las variables de entorno, para obtener la ruta de los datos. Si no
# la encuentra, se lanza una excepción.
load_dotenv('../.env')
if not getenv('DATA_SOURCE_PATH'):
    raise RuntimeError('Environment variable DATA_SOURCE_PATH not defined')
if not getenv('SAVED_PARQUET_PATH'):
    raise RuntimeError('Environment variable SAVED_PARQUET_PATH not defined')

rnd_state: int = 36  # Se define semilla para reproducibilidad

---

### CIC-DDoS2019

In [ ]:
datasets_loaded = [] # Lista para almacenar los datasets cargados

# Se define la ruta de origen de los datos en formato parquet
data_path = pjoin(getenv('DATA_SOURCE_PATH'), 'parquet', 'cic-ddos2019')

# Se cargan los datasets de la ruta especificada, en la lista datasets_loaded
print('Loading CIC-DDoS2019 datasets...')
for dataset_name in listdir(data_path):
    if dataset_name.endswith('.parquet'):
        datasets_loaded.append(pd.read_parquet(pjoin(data_path, dataset_name)))
        print(f'    -> CIC-DDoS2019 {dataset_name.split("-")[0]} - datos cargados')

# Luego, se concatenan los datasets cargados en un solo DataFrame y se muestra
# el shape (cantidad de filas y columnas) del DataFrame resultante.
df = pd.concat(datasets_loaded, ignore_index=True)
print(f'\nFinal shape: {df.shape}')

#### Previsualización de los datos

A continuación, se muestra una previsualización de los datos cargados, para
tener en cuenta las columnas y los valores que contiene.

In [ ]:
df.head(20)

#### Valores de la columna `Label`

Tal como se aprecia en la previsualización de los datos, la columna `Label`
es la que contiene la información de la etiqueta de los datos, y por lo
tanto, es la variable objetivo que se desea predecir. A continuación, se
presentan los valores únicos de esta columna, y su respectiva cantidad.

In [ ]:
df['Label'].value_counts()

Los datos presentan 18 etiquetas diferentes, las cuales plantean distintos
protocolos de red, usualmente victimas de ataques DDoS. En este contexto, se
plantea el uso de `LabelEncoder` para convertir la columna `Label` a valores
numericos, y asi poder entrenar los modelos de ML/DL.

In [ ]:
# Se aplica funcion lambda para convertir la columna 'Label' a binaria
# df['classification'] = df['Label'].apply(lambda x: 0 if x == 'Benign' else 1)

# Se usa LabelEncoder para convertir la columna 'Label' a numerica
le = LabelEncoder()
df['Label_encoded'] = le.fit_transform(df['Label'])

# Se muestra el mapeo de las clases de la columna 'Label' a valores numericos
for col_name, encoded_value in zip(le.classes_, le.transform(le.classes_)):
    print(f'- {col_name:13s} -> {encoded_value}')

In [ ]:
df.drop(columns=['Label'], inplace=True)  # Se elimina la columna 'Label'
df.head(20)  # Se muestra una previsualización de los datos

In [ ]:
df['Label_encoded'].value_counts()

Como se aprecia arriba, la columna `Label` ha sido convertida a valores
numericos, y puede ser usada para entrenamiento como variable objetivo.

#### Caracteristicas y valores nulos

En base a las columnas del DataFrame, se seleccionaran las caracteristicas
relevantes para el entrenamiento de los modelos. Ademas, se identificaran
aquellas columnas que contienen valores nulos, y se procedera a su
eliminacion del DataFrame.

In [ ]:
# Se muestran las columnas del DataFrame
df.columns

Los datos presentan caracteristicas importantes relacionadas con el trafico
de red, caracteristicas de los paquetes, entre otros. Se aprecia claramente
que los datos vienen correctamente segmentados, y que las columnas poseen
valores numericos que pueden ser utilizados para el entrenamiento de
algoritmos de ML/DL. En ese contexto, la documentacion del dataset contiene
cada columna y su respectiva descripcion, lo que facilita la seleccion de las
 caracteristicas que se adapten a los modelos propuestos.

In [ ]:
prev_shape = df.shape
# Se eliminan las columnas con valores nulos
df.dropna(axis=1, inplace=True)
print(f'{prev_shape} - {df.shape}')

#### Guardar DataFrame preprocesado

Una vez se obtiene la version final del DataFrame, se procede a guardarlo en
formato `.parquet` para su posterior uso en la etapa de entrenamiento de los
modelos

In [ ]:
save_parquet_path = pjoin(getenv('SAVED_PARQUET_PATH'), 'cic-ddos2019.parquet')
df.to_parquet(save_parquet_path)  # Se guarda el DataFrame en formato parquet

---

### N-BaIoT

In [ ]:
datasets_loaded = [] # Lista para almacenar los datasets cargados
nbaiot_csvs = ['benign', 'mirai', 'gafgyt']
nbaiot_protocols = ['scan', 'tcp', 'udp', 'ack', 'combo']

# Se define la ruta de origen de los datos en formato parquet
data_path = pjoin(getenv('DATA_SOURCE_PATH'), 'nbaiot')

# Se cargan los datasets de la ruta especificada, en la lista datasets_loaded
print('Loading N-BaIoT datasets...')

# Se cargan los datos benignos
for i in range(1, 10):
    _df = pd.read_csv(pjoin(data_path, f'{i}.benign.csv'))
    _df['malign'] = False
    datasets_loaded.append(_df)
    print(f'    -> N-BaIoT {i}.benign - datos cargados')

# Se recorren los datasets posibles de datos malignos
for i in range(1, 10):
    for botnet in nbaiot_csvs:
        for protocol in nbaiot_protocols:
            try:
                _df = pd.read_csv(pjoin(data_path, f'{i}.{botnet}'
                                                       f'.{protocol}.csv'))
            except FileNotFoundError:
                print(f'     X N-BaIoT {i}.{botnet}.{protocol} - no encontrado')
            else:
                _df['malign'] = True
                datasets_loaded.append(_df)
                print(f'    -> N-BaIoT {i}.{botnet}.{protocol} - datos cargados')

# Luego, se concatenan los datasets cargados en un solo DataFrame y se muestra
# el shape (cantidad de filas y columnas) del DataFrame resultante.
df = pd.concat(datasets_loaded, ignore_index=True)
print(f'\nFinal shape: {df.shape}')

#### Previsualización de los datos

A continuación, se muestra una previsualización de los datos cargados, para
tener en cuenta las columnas y los valores que contiene.

In [ ]:
df.head(20)  # Se muestra una previsualización de los datos

In [ ]:
df['malign'].value_counts()

#### Caracteristicas y valores nulos

En base a las columnas del DataFrame, se seleccionaran las caracteristicas
relevantes para el entrenamiento de los modelos. Ademas, se identificaran
aquellas columnas que contienen valores nulos, y se procedera a su
eliminacion del DataFrame.

In [ ]:
# Se muestran las columnas del DataFrame
df.columns

Los datos presentan caracteristicas importantes relacionadas con el trafico
de red, caracteristicas de los paquetes, entre otros. Se aprecia claramente
que los datos vienen correctamente segmentados, y que las columnas poseen
valores numericos que pueden ser utilizados para el entrenamiento de
algoritmos de ML/DL. En ese contexto, la documentacion del dataset contiene
cada columna y su respectiva descripcion, lo que facilita la seleccion de las
 caracteristicas que se adapten a los modelos propuestos.

In [ ]:
prev_shape = df.shape
# Se eliminan las columnas con valores nulos
df.dropna(axis=1, inplace=True)
print(f'{prev_shape} - {df.shape}')

#### Seleccion de caracteristicas

Se procedera a seleccionar las caracteristicas relevantes para el
entrenamiento, utilizando XGBoost y Random Forest.

In [ ]:
# Se establece uma muestra del dataframe para probar primero
df_sample = df.sample(n=100000, random_state=rnd_state)

# Se separa características y valor objetivo
X = df_sample.drop('malign', axis=1)
y = df_sample['malign']

# Se divide el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=rnd_state,
    stratify=y
)

##### Usando XGBoost

In [ ]:
# Inicializar y entrenar el modelo XGBoost
xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=rnd_state,
        scale_pos_weight=(sum(y_train == 0) / sum(y_train == 1)),  # Manejo de clases desbalanceadas
        use_label_encoder=False,
        eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)

# Obtener la importancia de las características
importances_xgb = xgb_model.feature_importances_
feature_names_xgb = X_train.columns

# Crear un DataFrame de importancias
feature_importances_xgb = pd.DataFrame({
    'feature': feature_names_xgb,
    'importance': importances_xgb
})

# Ordenar las características por importancia
feature_importances_xgb = feature_importances_xgb.sort_values(by='importance', ascending=False)

# Visualizar las 20 características más importantes
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importances_xgb.head(20))
plt.title('Top 20 Importancias de Características - XGBoost')
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.tight_layout()
plt.show()

# Seleccionar las características más importantes (por ejemplo, top 50)
top_features_xgb = feature_importances_xgb.head(50)['feature'].tolist()
print(f"Top 50 características según XGBoost: {top_features_xgb}")
df_xgb = df_sample[top_features_xgb + ['malign']]

##### Usando Random Forest

In [ ]:
# Inicializar y entrenar el modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=rnd_state, class_weight='balanced')
rf.fit(X_train, y_train)

# Obtener la importancia de las características
importances_rf = rf.feature_importances_
feature_names_rf = X_train.columns

# Crear un DataFrame de importancias
feature_importances_rf = pd.DataFrame({
    'feature': feature_names_rf,
    'importance': importances_rf
})

# Ordenar las características por importancia
feature_importances_rf = feature_importances_rf.sort_values(by='importance', ascending=False)

# Visualizar las 20 características más importantes
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importances_rf.head(20))
plt.title('Top 20 Importancias de Características - Random Forest')
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.tight_layout()
plt.show()

# Seleccionar las características más importantes (por ejemplo, top 50)
top_features_rf = feature_importances_rf.head(50)['feature'].tolist()
print(f"Top 50 características según Random Forest: {top_features_rf}")
df_rf = df_sample[top_features_rf + ['malign']]

#### Guardar DataFrame preprocesado

Una vez se obtiene la version final del DataFrame con la optimizacion de cada
 metodo, se procede a guardarlos en formato `.parquet` para su posterior uso
 en la etapa de entrenamiento de los modelos

In [ ]:
save_parquet_rf_path = pjoin(getenv('SAVED_PARQUET_PATH'), 'nbaiot-rf.parquet')
save_parquet_xgb_path = pjoin(getenv('SAVED_PARQUET_PATH'), 'nbaiot-xgb.parquet')

df_rf.to_parquet(save_parquet_rf_path)  # Se guarda el DataFrame en formato parquet
df_xgb.to_parquet(save_parquet_xgb_path)  # Se guarda el DataFrame en formato parquet